In [63]:
import glob, re, os, shutil, random
import pandas as pd
from pathlib import Path
import numpy as np

In [64]:
#############################################
######### Split label file E/I, ... #########
#############################################

TARGET_LIST = ['E/I', 'F/T', 'J/P', 'N/S']
for TARGET in TARGET_LIST:
    df = pd.read_csv("C:\\Users\\leeh9\\UGRP-MBTI\\2020-ugrp-Face-analysis-MBTI\\data\\label.csv", index_col= "No.")
    df.drop(df.columns[-3:], inplace=True, axis = 1)
    df.rename(columns={df.columns[0]:'MBTI'}, inplace=True)

    trait = TARGET.split('/')
    df2 = pd.DataFrame(columns=['filename', f'{trait[0]}', f'{trait[1]}'])

    os.chdir("C:\\Users\\leeh9\\UGRP-MBTI\\2020-ugrp-Face-analysis-MBTI\\data\\images")
    for file in glob.glob("*.jpg"):
            file_id = file.split('_')[0]
            personality = df.iloc[int(file_id)][TARGET].strip(' ')

            new_row = {
                    'filename' : file, 
                    f'{trait[0]}' : int(personality == trait[0]), 
                    f'{trait[1]}' : int(personality == trait[1])
            }
            df2 = df2.append(new_row, ignore_index=True)
    os.chdir("C:\\Users\\leeh9\\UGRP-MBTI\\2020-ugrp-Face-analysis-MBTI\\data")
    df2.to_csv(f'label_{trait[0]}_{trait[1]}.csv', index=False)

In [5]:
###################################
##### Train, Validation Split #####
###################################

#########################
CLASS = ('E', 'I')
#########################

# Create missing directory
for dir in ["train", "validation"]:
    for c in CLASS:
        Path(f"data/EI/{dir}/{CLASS[0]}").mkdir(parents=True, exist_ok=True)
        Path(f"data/EI/{dir}/{CLASS[1]}").mkdir(parents=True, exist_ok=True)
df = pd.read_csv(f"data/label_{CLASS[0]}_{CLASS[1]}.csv" )

# lambda function for PATH setting
TARGET_PATH = lambda target, cl : f"data/{CLASS[0]}{CLASS[1]}/{target}/{cl}"
SOURCE_PATH = lambda name : f"data/renamed/{name}"

# iterate over index and copy data from source path
for index, data in df.iterrows():
    t = "validation" if index//5 +1 in VAL_LIST else "train"
    if data.loc[CLASS[0]]==1:
        target_class = CLASS[0]
    else:
        target_class = CLASS[1]
    try:
        shutil.copy(SOURCE_PATH(data["filename"]), TARGET_PATH(t, target_class))
    except:
        pass


